In [ ]:
'''
一些说明：
私有函数：函数名前是两个下划线__，说明仅供后端调用，不开放给前端
'''

In [28]:
import pandas as pd
import random

# 获取小题题目类型函数
# 返回一个字典
def __getProblemType_dict():
    df_ProblemType=pd.read_excel('excel_all.xlsx',sheet_name='ProblemType')
    ProblemType_dict={}
    for i in range(0,len(df_ProblemType)):
        TypeNum=df_ProblemType.loc[i,'TypeNum'] # i行TypeNum列
        TypeName=df_ProblemType.loc[i,'TypeName']
        ProblemType_dict[TypeNum]=TypeName
    return ProblemType_dict


In [ ]:
# 大题类型，字典存储
PROBLEM_SET_DICT={
    'Basic python':'BasicPython',
    'loop&logic':'LoopAndLogic', 
    'Function':'Function',
    'Virtualization':'Virtualization',
    'Nump& Pandas&others':'NumpAndPandas'
}


In [32]:
#小题对象：单选、判断、多选、填空
class SingleChoice:
    def __init__(self,problemid,problemTypeNum,problem):
        self.problemid=problemid
        self.problemTypeNum=problemTypeNum
        self.problem=problem

    def set_choice(self,A,B,C,D):
        self.A=A
        self.B=B
        self.C=C
        self.D=D
    
    def __str__(self):#打印print(对象)
        return "题号：%d 题型：单选题 题目：%s A.%s B.%s C.%s D.%s" % (self.problemid,self.problem,self.A,self.B,self.C,self.D)

class MultipleChoice:
    def __init__(self,problemid,problemTypeNum,problem):
        self.problemid=problemid
        self.problemTypeNum=problemTypeNum
        self.problem=problem

    def set_choice(self,A,B,C,D):
        self.A=A
        self.B=B
        self.C=C
        self.D=D
    
    def __str__(self):
        return "题号：%d 题型：多选题 题目：%s A.%s B.%s C.%s D.%s" % (self.problemid,self.problem,self.A,self.B,self.C,self.D)
       
class Judge:
    def __init__(self,problemid,problemTypeNum,problem):
        self.problemid=problemid
        self.problemTypeNum=problemTypeNum
        self.problem=problem

    def set_choice(self,A,B):
        self.A=A
        self.B=B

    def __str__(self):
        return "题号：%d 题型：判断题 题目：%s A.%s B.%s" % (self.problemid,self.problem,self.A,self.B)
        

class FillBlanks:
    def __init__(self,problemid,problemTypeNum,problem):
        self.problemid=problemid
        self.problemTypeNum=problemTypeNum
        self.problem=problem
    
    def __str__(self):
        return "题号：%d 题型：填空题 题目：%s" % (self.problemid,self.problem)

class CodeProblem:
    def __init__(self,problemid,problemTypeNum,problem):
        self.problemid=problemid
        self.problemTypeNum=problemTypeNum
        self.problem=problem
    
    def __str__(self):
        return "题号：%d 题型：填空题 题目：%s" % (self.problemid,self.problem)

class Answer:
    def __init__(self,problemid,type,answer):
        self.problemid=problemid
        self.type=type      
        self.answer=answer

    def __str__(self):
        return f"题号：{self.problemid} 题型：{self.type} 答案：{self.answer}"

In [54]:
# 从pandas的dataframe构造小题对象的构造函数
def __build_SingleChoice(df_problem_row):
    singleChoice = SingleChoice(df_problem_row['problemid'],df_problem_row['type'],df_problem_row['problem'])
    singleChoice.set_choice(df_problem_row['A'],df_problem_row['B'],df_problem_row['C'],df_problem_row['D'])
    return singleChoice

def __build_MultipleChoice(df_problem_row):
    multipleChoice = MultipleChoice(df_problem_row['problemid'],df_problem_row['type'],df_problem_row['problem'])
    multipleChoice.set_choice(df_problem_row['A'],df_problem_row['B'],df_problem_row['C'],df_problem_row['D'])
    return multipleChoice

def __build_Judge(df_problem_row):
    judge = Judge(df_problem_row['problemid'],df_problem_row['type'],df_problem_row['problem'])
    judge.set_choice(df_problem_row['A'],df_problem_row['B'])
    return judge

def __build_FillBlanks(m,problem_index,problemid,problemTypeNum,problem,df_problem_all):
    pass

def __CodeProblem_None(m,problem_index,problemid,problemTypeNum,problem,df_problem_all):
    pass


# 这个函数给前端调用，返回一个大题的所有小题
# 前端提供输入参数：problem_set 大题名称，后端是表名
# 返回值：小题对象的列表
def build_problem_list(problem_set):
    # 读问题表：
    sheet_choose=PROBLEM_SET_DICT[problem_set]
    #提取该题库对应的sheet
    df_problem_all=pd.read_excel('excel_all.xlsx',sheet_name=sheet_choose)
    result_list=[]
    for index,row in df_problem_all.iterrows():
        problem_type = row['type']
        if problem_type==1:  # 要改
            result_list.append(__build_SingleChoice(row))
        elif problem_type==2:
            result_list.append(__build_Judge(row))
        elif problem_type==3:
            result_list.append(__build_MultipleChoice(row))
        elif problem_type==4:
            result_list.append(__build_FillBlanks(row))
        else:
            continue
    return result_list

In [55]:
#从excel中取出存储的答案,存在字典中
# 返回值：字典：key是题号，value是answer对象
def __read_answer():
    df_answer=pd.read_excel('excel_all.xlsx',sheet_name='Answer')
    answer_list = dict()
    for indwx, row in df_answer.iterrows():
        answer_list[row['problemid']] = Answer(row['problemid'],row['type'],row['answer'])
    return answer_list

# 前端调用的函数：用户选择和答案匹配
# 输入参数：name：用户名称；user_choice: {题号:'用户选择'}
def match_answer(name, user_choice):
    answer_list = __read_answer()
    right = dict()
    wrong = dict()
    for k,v in user_choice.items():
        answer = answer_list[k]
        type = answer.type
        if type==3:
            v = sorted(v)
        if v==answer.answer:
            right[k]=type
        else:
            wrong[k]=type
    # 把right和wrong写表

In [ ]:
#随机选取5道题
#生成5个随机数
row_len=df_problem_all.shape[0]
lista=random.sample(range(0,row_len),5)
lista=sorted(lista)

In [3]:
import pandas as pd
df_user = pd.read_excel('excel_all.xlsx', sheet_name='User')

df_user

,ID,name,passwd,right,wrong
0,NaN,test_user,12345.0,"2,3,4","a,b,c"
1,NaN,test_u2,NaN,"2,3,4","a,b,c"


In [1]:
df_right = pd.DataFrame({'right': ['1,2,3'],'name':['test_user']})

for i,r in df_user.iterrows():
    if r['name']=='test_user':
        df_user.iloc[i,3]=['1,2,3']
    

df_user

NameError: name 'pd' is not defined

In [18]:
writer = pd.ExcelWriter('excel_all.xlsx')
df_user.to_excel(writer, sheet_name='User',index=False)# index = False表示不写入索引
writer.save()

In [21]:
l=','.join(['1','2'])
l

'1,2'

In [17]:
from openpyxl import load_workbook

def write_user_right(name, right, wrong):
    df_user = pd.read_excel('excel_all.xlsx', sheet_name='User')
    
    wb=load_workbook(r'excel_all.xlsx') #打开指定excel表
    sheet=wb["User"]
    for index, row in df_user.iterrows():
        if row['name'] == name:
            sheet.cell(index+2,4,str(right))
            sheet.cell(index+2,5,str(wrong))
            break
    wb.save(r"excel_all.xlsx")

In [19]:
write_user_right('test_user','2,3,4','a,b,c')

In [21]:
import xlwings as xw

wb = xw.Book('excel_all.xlsx')
sheet = xw.books['excel_all.xlsx'].sheets['User']
# #在wb中新建一张新的sheet.可以指定位置
# sht = wb.sheets.add(name="User",before=None,after=None)
# #df.values 不然会插入df的索引
# sht.range("A1").value = df.values
rng = sheet.range('a1')
rng
wb.save()
wb.close()

In [20]:
a = sorted('CA')
a

['A', 'C']